In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc

import pandas as pd
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# from torch.optim import AdamW
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel #AutoModelForSequenceClassification
# from transformers import get_linear_schedule_with_warmup
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score

# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
#     sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     return sum_embeddings / sum_mask

In [ ]:
# sectors = ['finance', 'gold', 'ferrous_metallurgy', 'oil_gas', 'non_ferrous_metallurgy', 'electrical networks', 'telecom', 'it', 'real_estate']
companies = ['AFLT', 'VTBR', 'GAZP', 'GMKN', 'LKOH', 'ROSN', 'NVTK', 'SBER', 'TCSG', 'YNDX']

## Получаем эмбеддинги Берта для классического ML

In [ ]:
class NewsDatasetEmb(Dataset):
    def __init__(self, texts, tokenizer):

      self.texts = [tokenizer(text,
                          padding='max_length', max_length = 256, truncation=True,
                          return_tensors="pt") for text in texts]

    def __len__(self):
        return len(self.texts)


    def __getitem__(self, idx):

        batch_texts = self.texts[idx]

        return batch_texts

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Нейросети для исследований научка/data.parquet')

In [ ]:
df['message_raw'].apply(lambda x: len(x.split())).quantile(0.9)

221.0

In [ ]:
class BertEmbeddings(nn.Module):
  def __init__(self):
    super(BertEmbeddings, self).__init__()

    self.bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

  def forward(self, input_ids, attention_mask):
      outputs = self.bert(input_ids, attention_mask)
      return outputs[0][:,0,:]

In [ ]:
bert = BertEmbeddings().cuda()

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
texts = df.message_raw.values
dataset = NewsDatasetEmb(texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False, drop_last=False)

In [ ]:
device = 'cuda'
all_data = []
for batch in tqdm(dataloader):
  input_ids = batch['input_ids'].squeeze(1).to(device)
  masks = batch['attention_mask'].squeeze(1).to(device)
  data = bert(input_ids, masks).detach().to('cpu').tolist()
  all_data.append(data)

npy = np.concatenate(all_data)
npy.save('/content/drive/MyDrive/Нейросети для исследований научка/bert_embs.npy', npy)

100%|██████████| 13970/13970 [30:25<00:00,  7.65it/s]


## То что осталось от диссера (пока не используется)

In [ ]:
# class BertClassifier(nn.Module):
#     def __init__(self, freeze=False):
#         super(BertClassifier, self).__init__()

#         input_layer = 768
#         hidden_layer = 50
#         output_layer = 3

#         self.bert = AutoModel.from_pretrained("DeepPavlov/distilrubert-base-cased-conversational")

#         self.fc1 = nn.Linear(input_layer*2, input_layer//2)
#         self.fc2 = nn.Linear(input_layer*2 + input_layer//2, input_layer)
#         self.fc3 = nn.Linear(input_layer, output_layer)

#         self.drop1 = nn.Dropout(0.1)
#         self.drop2 = nn.Dropout(0.1)

#         self.act = nn.ReLU()

#         if freeze:
#             for param in self.bert.parameters():
#                 param.requires_grad = False

#     def forward(self, input_ids, attention_mask):
#         outputs = self.bert(input_ids, attention_mask)
#         sentence_embeddings = mean_pooling(outputs[1], attention_mask)
#         embeddings = torch.cat([outputs[0][:,0,:], sentence_embeddings], dim=1)
#         embeddings1 = self.fc1(embeddings)
#         embeddings = torch.cat([embeddings, embeddings1], dim=1)
#         embeddings = self.act(embeddings)
#         embeddings = self.drop1(embeddings)
#         embeddings = self.fc2(embeddings)
#         embeddings = self.act(embeddings)
#         embeddings = self.drop2(embeddings)
#         logits = self.fc3(embeddings)

#         return logits

In [ ]:
def calculate_metrics(labels, preds, preds_bin):
    acc = accuracy_score(labels, preds_bin)
    pre = precision_score(labels, preds_bin)
    rec = recall_score(labels, preds_bin)
    f1 = f1_score(labels, preds_bin)
    auc = roc_auc_score(labels, preds[:,1])
    kappa = cohen_kappa_score(labels, preds_bin)
    return acc, pre, rec, f1, auc, kappa

In [ ]:
def train(epochs, train_dataloader, val_dataloader):
    best_acc = -np.inf

    for e in range(epochs):
        model.train()

        train_loss = 0

        for batch in tqdm(train_dataloader):

            batch_data, batch_labels = batch

            batch_inputs = batch_data['input_ids'].to(device).squeeze(1)
            batch_masks = batch_data['attention_mask'].to(device).squeeze(1)
            batch_labels = batch_labels.to(device)

            model.zero_grad()

            logits = model(batch_inputs, batch_masks)
            loss = loss_function(logits, batch_labels)

            loss.backward()

            train_loss += loss.item()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        train_loss /= len(train_dataloader)

        model.eval()

        test_loss = 0

        all_test = []
        all_test_bin = []
        labels = []

        for batch in tqdm(val_dataloader):

            batch_data, batch_labels = batch

            batch_inputs = batch_data['input_ids'].to(device).squeeze(1)
            batch_masks = batch_data['attention_mask'].to(device).squeeze(1)
            batch_labels = batch_labels.to(device)

            with torch.no_grad():
                logits = model(batch_inputs, batch_masks)

            loss = loss_function(logits, batch_labels)
            test_loss += loss.item()

            preds = F.softmax(logits, dim=-1)
            preds_bin = preds.argmax(dim=-1) - 1

            batch_labels = batch_labels.detach().cpu().numpy()
            preds_bin = preds_bin.detach().cpu().numpy()
            preds = preds.detach().cpu().numpy()

            all_test.append(preds)
            all_test_bin.append(preds_bin)
            labels.append(batch_labels)

        test_loss /= len(val_dataloader)
        all_test = np.concatenate(all_test, axis=0)
        all_test_bin = np.concatenate(all_test_bin, axis=0)
        labels = np.concatenate(labels, axis=0)

        acc, pre, rec, f1, auc, kappa = calculate_metrics(labels, all_test, all_test_bin)

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), f'/content/drive/MyDrive/weights/dbert_tg_{col}_bin.pth')

        print('Epoch: %d  |  Train Loss: %1.5f  |  Test Loss: %1.5f'%(e+1, train_loss, test_loss))

        print('Val acc: %1.4f'%(acc))
        print('Val pre: %1.4f'%(pre))
        print('Val rec: %1.4f'%(rec))
        print('Val f1: %1.4f'%(f1))
        print('Val auc: %1.4f'%(auc))
        print('Val kappa: %1.4f'%(kappa))

In [ ]:
def evaluate(test_dataloader):

  model.load_state_dict(torch.load(f'/content/drive/MyDrive/weights/dbert_tg_{col}_bin.pth'))
  model.eval()

  test_loss = 0

  all_test = []
  all_test_bin = []
  labels = []

  for batch in tqdm(test_dataloader):

      batch_data, batch_labels = batch

      batch_inputs = batch_data['input_ids'].to(device).squeeze(1)
      batch_masks = batch_data['attention_mask'].to(device).squeeze(1)
      batch_labels = batch_labels.to(device)

      with torch.no_grad():
          logits = model(batch_inputs, batch_masks)

      loss = loss_function(logits, batch_labels)
      test_loss += loss.item()

      preds = F.softmax(logits, dim=-1)
      preds_bin = preds.argmax(dim=-1) - 1

      batch_labels = batch_labels.detach().cpu().numpy()
      preds_bin = preds_bin.detach().cpu().numpy()
      preds = preds.detach().cpu().numpy()

      all_test.append(preds)
      all_test_bin.append(preds_bin)
      labels.append(batch_labels)

  test_loss /= len(test_dataloader)
  all_test = np.concatenate(all_test, axis=0)
  all_test_bin = np.concatenate(all_test_bin, axis=0)
  labels = np.concatenate(labels, axis=0)

  acc, pre, rec, f1, auc, kappa = calculate_metrics(labels, all_test, all_test_bin)

  print('Test loss: %1.4f'%(test_loss))
  print('Test acc: %1.4f'%(acc))
  print('Test pre: %1.4f'%(pre))
  print('Test rec: %1.4f'%(rec))
  print('Test f1: %1.4f'%(f1))
  print('Test auc: %1.4f'%(auc))
  print('Test kappa: %1.4f'%(kappa))
  return acc, pre, rec, f1, auc, kappa

In [ ]:
news = pd.read_parquet('/content/drive/MyDrive/Диссертация/Парсеры сайтов/tg_comps_raw_1,5sigma_.parquet')
# news['message'] = news['title'] + ' ' + news['announce']
# news = news[news['message'] != ' '].copy()
# news.drop(columns=['link', 'title', 'announce', 'text'], inplace=True)
news.dropna(subset=['message'],inplace=True)

news_train = news[news.date < '2021-01-01'].copy()
news_val = news[(news.date > '2021-01-01') & (news.date < '2021-06-01')].copy()
news_test = news[(news.date < '2022-01-01') & (news.date > '2021-06-01')].copy()

In [ ]:
loggs = pd.DataFrame()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for company in ['LSRG', 'DSKY', 'MOEX', 'MTSS']:
for company in ['NVTK']:
    print(company)

    first_dates = pd.read_csv(f'/content/drive/MyDrive/Диссертация/Парсеры сайтов/Стоимость акций/1 мин/first_dates.csv', index_col='company')
    first_day = pd.to_datetime(first_dates.loc[company,:].values[0])

    comp_train = news_train[news_train[company] == True].copy()
    comp_val = news_val[news_val[company] == True].copy()
    comp_test = news_test[news_test[company] == True].copy()

    cols = ['date', 'message'] + [col for col in news_train.columns if f'{company}_' in col]
    cols = [col for col in comp_test.columns if f'{company}_' in col]

    train_texts, val_texts, test_texts = comp_train['message'].values, comp_val['message'].values, comp_test['message'].values

    for col in cols[2:]:
      print(col)
      train_target, val_target, test_target = comp_train[col].values, comp_val[col].values, comp_test[col].values

      batch_size = 32
      train_dataset, val_dataset, test_dataset = NewsDataset(train_texts, train_target), NewsDataset(val_texts, val_target), NewsDataset(test_texts, test_target)

      train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
      val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
      test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

      model = BertClassifier(freeze=False).to(device)

      epochs = 8
      steps = len(train_dataloader) * epochs
      learning_rate = 5e-5
      epsilon = 1e-8

      loss_function = nn.CrossEntropyLoss()
      optimizer = AdamW(model.parameters(), lr=learning_rate, eps=epsilon)
      scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=steps)

      train(epochs, train_dataloader, val_dataloader)
      acc, pre, rec, f1, auc, kappa = evaluate(test_dataloader)

      loggs[f'{col}'] = [acc, pre, rec, auc, f1, kappa]
      loggs.to_csv('/content/drive/MyDrive/Диссертация/tg_loggs/dist_bert_tg_loggs_neutral_magn_chmf_dsky_sngs_nvtk.csv')

      del model
      del train_dataset
      del val_dataset
      del test_dataset
      del train_dataloader
      del val_dataloader
      del test_dataloader

      gc.collect()
      torch.cuda.empty_cache()
      gc.collect()

NVTK
NVTK_10_min_mean


Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 19/19 [00:10<00:00,  1.89it/s]


Epoch: 1  |  Train Loss: 0.69544  |  Test Loss: 0.69522
Val acc: 0.5329
Val pre: 0.5329
Val rec: 1.0000
Val f1: 0.6953
Val auc: 0.4701
Val kappa: 0.0000


100%|██████████| 19/19 [00:10<00:00,  1.82it/s]


Epoch: 2  |  Train Loss: 0.69031  |  Test Loss: 0.70761
Val acc: 0.5329
Val pre: 0.5329
Val rec: 1.0000
Val f1: 0.6953
Val auc: 0.4634
Val kappa: 0.0000


...

Epoch: 7  |  Train Loss: 0.62756  |  Test Loss: 0.74262
Val acc: 0.4934
Val pre: 0.4657
Val rec: 0.5739
Val f1: 0.5142
Val auc: 0.5022
Val kappa: -0.0032


100%|██████████| 19/19 [00:10<00:00,  1.81it/s]


Epoch: 8  |  Train Loss: 0.51968  |  Test Loss: 0.85978
Val acc: 0.5082
Val pre: 0.4681
Val rec: 0.3873
Val f1: 0.4239
Val auc: 0.4927
Val kappa: 0.0015


100%|██████████| 31/31 [00:16<00:00,  1.87it/s]


Test loss: 0.6949
Test acc: 0.4732
Test pre: 0.5589
Test rec: 0.3040
Test f1: 0.3938
Test auc: 0.4762
Test kappa: -0.0046
